In [2]:
%pip install --upgrade --quiet  python-dotenv text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [10]:
import dotenv 
import os

dotenv.load_dotenv()


os.environ["DEBUG"] = "1"  # Set to "1" if you want to use debug mode.
os.environ["HUGGINGFACEHUB_API_TOKEN"] = os.getenv("HUGGINGFACEHUB_API_TOKEN")


<bound method HfFolder.save_token of <class 'huggingface_hub.utils._hf_folder.HfFolder'>>

In [16]:
from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate

from langchain.output_parsers import CommaSeparatedListOutputParser

from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models.huggingface import ChatHuggingFace

from huggingface_hub.hf_api import HfFolder
HfFolder.save_token

<bound method HfFolder.save_token of <class 'huggingface_hub.utils._hf_folder.HfFolder'>>

In [17]:
llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 512,
        "top_k": 30,
        "temperature": 0.1,
        "repetition_penalty": 1.03,
    },
)

print(llm)

HuggingFaceHub
Params: {'repo_id': 'HuggingFaceH4/zephyr-7b-beta', 'task': 'text-generation', 'model_kwargs': {'max_new_tokens': 512, 'top_k': 30, 'temperature': 0.1, 'repetition_penalty': 1.03}}


In [18]:
# from langchain.schema import (
#     HumanMessage,
#     SystemMessage,
# )
# from langchain_community.chat_models.huggingface import ChatHuggingFace

# messages = [
#     SystemMessage(content="Explain the meaning and origin of a user-provided idiom or proverb, including its figurative meaning, typical usage, historical context, and any interesting origin stories"),
#     HumanMessage(
#         content="Break a leg?"
#     ),
# ]

# chat_model = ChatHuggingFace(llm=llm)

In [24]:
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_model = ChatHuggingFace(llm=llm)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=(
                "Explain the meaning and origin of a user-provided idiom or proverb, including its figurative meaning, typical usage, historical context, and any interesting origin stories"
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

chain = prompt | chat_model

messages = prompt.format_messages(text="Once in blue moon")

print(chain)



first=ChatPromptTemplate(input_variables=['text'], messages=[SystemMessage(content='Explain the meaning and origin of a user-provided idiom or proverb, including its figurative meaning, typical usage, historical context, and any interesting origin stories'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='{text}'))]) last=ChatHuggingFace(llm=HuggingFaceHub(client=<InferenceClient(model='mistralai/Mistral-7B-v0.1', timeout=None)>, repo_id='mistralai/Mistral-7B-v0.1', task='text-generation', model_kwargs={'max_new_tokens': 1024, 'top_k': 30, 'temperature': 0.9, 'repetition_penalty': 1.03}), tokenizer=LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rst

In [21]:
#Inspect how the chat messages are formatted for the LLM call.
chat_model._to_chat_prompt(messages)

'<|system|>\nExplain the meaning and origin of a user-provided idiom or proverb, including its figurative meaning, typical usage, historical context, and any interesting origin stories</s>\n<|user|>\nOnce in blue moon</s>\n<|assistant|>\n'

In [22]:
res = chat_model.invoke(messages)
print(res.content)

<|system|>
Explain the meaning and origin of a user-provided idiom or proverb, including its figurative meaning, typical usage, historical context, and any interesting origin stories</s>
<|user|>
Once in blue moon</s>
<|assistant|>
The idiom "once in a blue moon" refers to a rare or infrequent occurrence. The phrase originated from the ancient Roman belief that a full moon appeared once every 29 or 30 days, making it a rare sight when it coincided with a second full moon in a calendar month, which happens approximately every 2.7 years. This second full moon was called a "blue moon," as it was believed that the moon appeared blue during this rare occurrence.

However, the modern usage of the term "blue moon" has evolved to mean simply an extremely rare occurrence, and does not necessarily refer to a blue-colored moon. In fact, the moon never actually appears blue, as its color is determined by the amount of dust and pollution in the atmosphere, not by the phase of the moon.

The express

In [22]:
llm = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    task="text-generation",
    model_kwargs={
        "max_new_tokens": 1024,
        "top_k": 30,
        "temperature": 0.9,
        "repetition_penalty": 1.03,
    },
)

In [23]:
output_parser = CommaSeparatedListOutputParser()


chat = ChatHuggingFace(llm=llm)

prompt = ChatPromptTemplate.from_template("Explain the meaning and origin of the user-provided idiom or proverb {input}, including its figurative meaning, typical usage, historical context, and any interesting origin stories.")
chain = prompt | chat |output_parser


chain.invoke({"input": "Once in a blue moon"})




tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



['<s>[INST] Explain the meaning and origin of the user-provided idiom or proverb Once in a blue moon',
 'including its figurative meaning',
 'typical usage',
 'historical context',
 'and any interesting origin stories. [/INST] It could be anything like a phrase from a poem',
 'an expression of a person',
 'an old saying',
 'or a proverb.\n\nOnce in a blue moon:\n\n1. Figurative meaning:\nIt’s an English idiom that basically means “not often”.\n\n2. Typical Usage:\n– Once in a blue moon',
 'I go out for coffee with my friends. [Not often]\n– That person is once in a blue moon.\n– He is such a nice guy; he helps me every once in a blue moon.\n\n3. Historical Context:\nIt originated from a folktale according to which',
 'it used to rain gold for a night when a blue moon appeared in the sky. The phrase came into being as a reference to this phenomenon and it was considered an indicator of good fortune.\n\nHowever',
 'the term was not widely used until the 1940s. The expression was first us